In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join

I downloaded exoplanet host stars from here:
https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PSCompPars

In [2]:
?Table.read

In [3]:
nexsci = Table.read("nexsci2022-06-13.csv", format="ascii.csv", comment="#")

In [4]:
len(nexsci), len(nexsci.colnames)

(5044, 142)

In [5]:
nexsci

loc_rowid,pl_name,hostname,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_facility,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassprov,pl_dens,pl_denserr1,pl_denserr2,pl_denslim,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,ttv_flag,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_lum,st_lumerr1,st_lumerr2,st_lumlim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_age,st_ageerr1,st_ageerr2,st_agelim,st_dens,st_denserr1,st_denserr2,st_denslim,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,st_rotp,st_rotperr1,st_rotperr2,st_rotplim,st_radv,st_radverr1,st_radverr2,st_radvlim,ra,dec,sy_dist,sy_disterr1,sy_disterr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_jmag,sy_jmagerr1,sy_jmagerr2,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2
int64,str29,str27,str11,str12,str14,str28,int64,int64,int64,str29,int64,str178,str46,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,str16,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,int64,int64,str16,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,str6,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64
1,11 Com b,11 Com,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,0,Radial Velocity,2007,<a refstr=LIU_ET_AL__2008 href=https://ui.adsabs.harvard.edu/abs/2008ApJ...672..553L/abstract target=ref> Liu et al. 2008 </a>,Xinglong Station,0,326.03,0.32,-0.32,0,1.29,0.05,-0.05,0,12.1,--,--,0,1.08,--,--,0,6165.6,476.7,-476.7,0,19.4,1.5,-1.5,0,Msini,19.1,--,--,0,0.231,0.005,-0.005,0,--,--,--,--,--,--,--,--,--,--,--,--,0,G8 III,4742.0,100.0,-100.0,0,19.0,2.0,-2.0,0,2.7,0.3,-0.3,0,-0.35,0.09,-0.09,0,[Fe/H],2.243,0.071,-0.085,0,2.31,0.1,-0.1,0,--,--,--,--,--,--,--,--,1.2,1.0,-1.0,0,--,--,--,--,43.37,0.21,-0.21,0,185.1787793,17.7932516,93.1846,1.9238,-1.9238,5.726,0.017,-0.017,4.72307,0.023,-0.023,2.943,0.334,-0.334,2.484,0.268,-0.268,2.282,0.346,-0.346,0.639,--,--,0.732,--,--,4.44038,0.0038479,-0.0038479,3.8379,0.0071,-0.0071,--,--,--
2,11 UMi b,11 UMi,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,0,Radial Velocity,2009,<a refstr=DOLLINGER_ET_AL__2009 href=https://ui.adsabs.harvard.edu/abs/2009A&A...505.1311D/abstract target=ref> Dollinger et al. 2009 </a>,Thueringer Landessternwarte Tautenburg,0,516.21997,3.2,-3.2,0,1.53,0.07,-0.07,0,12.3,--,--,0,1.09,--,--,0,4684.8142,794.575,-794.575,0,14.74,2.5,-2.5,0,Msini,13.8,--,--,0,0.08,0.03,-0.03,0,--,--,--,--,--,--,--,--,--,--,--,--,0,K4 III,4213.0,46.0,-46.0,0,29.79,2.84,-2.84,0,2.78,0.69,-

The IDs here are DR2, so we'll have to crossmatch to DR3 IDs on the catalog. Fortunately they provide a table, but we'll update to make a column stripping the "Gaia DR2" string.

Note that it's a bad idea to use SIMBAD's coordinates, because most planet stars are nearby and have large proper motions.

In [6]:
# some fanciness that broke
# nexsci["dr2_source_id"] = [np.int64(x.split()[2]) for x in nexsci["gaia_id"]]


In [7]:
nexsci["gaia_id"][0]

'Gaia DR2 3946945413106333696'

In [8]:
# some fanciness
def get_source_id(x):
    try:
        return np.int64(x.split()[2])
    except AttributeError as e: # these are things where there is no Gaia ID for whatever reason, maybe too bright
        #print(x,e)
        return -1
    except IndexError as e: # these are things named "Gaia-1" or something
        print(x,e)
        return -1
nexsci["dr2_source_id"] = [get_source_id(x) for x in nexsci["gaia_id"]]

Gaia-1 list index out of range
Gaia-2 list index out of range


What are these missing planets?

In [9]:
for x,y in zip(*np.unique(nexsci["discoverymethod"], return_counts=True)):
    print(x,y)

Astrometry 1
Disk Kinematics 1
Eclipse Timing Variations 18
Imaging 60
Microlensing 130
Orbital Brightness Modulation 9
Pulsar Timing 7
Pulsation Timing Variations 2
Radial Velocity 930
Transit 3864
Transit Timing Variations 22


In [10]:
ii = nexsci["dr2_source_id"]==-1
for x,y in zip(*np.unique(nexsci["discoverymethod"][ii], return_counts=True)):
    print(x,y)

Eclipse Timing Variations 1
Imaging 5
Microlensing 129
Pulsar Timing 6
Radial Velocity 12
Transit 35


In [11]:
nexsci[nexsci["dr2_source_id"]==-1].show_in_notebook()

idx,loc_rowid,pl_name,hostname,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_facility,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassprov,pl_dens,pl_denserr1,pl_denserr2,pl_denslim,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,ttv_flag,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_lum,st_lumerr1,st_lumerr2,st_lumlim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_age,st_ageerr1,st_ageerr2,st_agelim,st_dens,st_denserr1,st_denserr2,st_denslim,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,st_rotp,st_rotperr1,st_rotperr2,st_rotplim,st_radv,st_radverr1,st_radverr2,st_radvlim,ra,dec,sy_dist,sy_disterr1,sy_disterr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_jmag,sy_jmagerr1,sy_jmagerr2,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,dr2_source_id
0,13,2MASS J01033563-5515561 AB b,2MASS J01033563-5515561 A,--,--,TIC 206502540,--,2,1,1,Imaging,2013,<a refstr=DELORME_ET_AL__2013 href=https://ui.adsabs.harvard.edu/abs/2013A&A...553L...5D/abstract target=ref>Delorme et al. 2013</a>,Paranal Observatory,0,--,--,--,--,84.0,--,--,0,12.3,--,--,0,1.1,--,--,0,4131.79,317.83,-317.83,0,13.0,1.0,-1.0,0,Mass,12.2,--,--,0,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0,M5.5/M6,--,--,--,--,--,--,--,--,0.19,0.02,-0.02,0,--,--,--,--,--,--,--,--,--,--,--,--,--,0.03,--,--,0,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,15.8992489,-55.2658291,47.2,3.1,-3.1,16.993,0.121,-0.121,15.788,0.275,-0.275,10.162,0.023,-0.023,9.58,0.025,-0.025,9.237001,0.022,-0.022,9.047,0.023,-0.023,8.804,0.02,-0.02,--,--,--,12.089,0.017,-0.017,--,--,--,-1
1,75,CFBDSIR J145829+101343 b,CFBDSIR J145829+101343,--,--,--,--,1,1,0,Imaging,2011,<a refstr=LIU_ET_AL__2011 href=https://ui.adsabs.harvard.edu/abs/2011ApJ...740..108L/abstract target=ref> Liu et al. 2011 </a>,W. M. Keck Observatory,0,10037.5,2737.5,-2737.5,0,2.6,0.3,-0.3,0,12.4,--,--,0,1.11,--,--,0,3337.1,1430.2,-1430.2,0,10.5,4.5,-4.5,0,Mass,9.62,--,--,0,--,--,--,--,--,--,--,--,370,40,-40,0,--,--,--,--,0,--,580.5,24.5,-24.5,0,--,--,--,--,0.02,0.01,-0.01,0,--,--,--,--,--,--,--,--,--,4.72,0.28,-0.28,0,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,224.6208333,10.2286111,23.1,2.4,-2.4,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,-1
2,76,CFHTWIR-Oph 98 b,CFHTWIR-Oph 98 A,--,--,TIC 175744063,--,1,1,0,Imaging,2021,<a refstr=FONTANIVE_ET_AL__2020 href=https://ui.adsabs.harvard.edu/abs/2020ApJ...905L..14F/abstract target=ref>Fontanive et al. 2020</a>,Hubble Space Telescope,0,8040000.0,470000.0,-470000.0,0,200.0,6.0,-6.0,0,20.849,0.56,-0.56,0,1.86,0.05,-0.05,0,2479.06157,222.47988,-254.26273,0,7.8,0.7,-0.8,0,Mass,1.5,--,--,0,--,--,--,--,--,--,--,--,1800,40,-40,0,--,--,--,--,0,M9-L1,2320.0,40.0,-40.0,0,0.24,0.01,-0.01,0,0.01,0.0,0.0,0,--,--,--,--,--,-2.85,0.06,-0.06,0,3.85,0.01,-0.01,0,0.003,--,--,0,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,246.934277,-23.98115,--,--,--,--,--,--,--,--,--,16.775,0.176,-0.176,15.574,0.109,-0.109,14.593,0.098,-0.098,13.99,0.044,-0.044,13.231,0.037,-0.037,--,--,--,18.525,1.0,-1.0,--,--,--,-1
3,123,DMPP-3 A b,DMPP-3 A,HD 42936,HIP 28941,--,--,2,1,0,Radial Velocity,2019,<a refstr=BARNES_ET_AL__2020 href=https://ui.adsab

A lot of the missing planets are microlensing, so it makes sense they're not in here.
Not sure why other objects don't have Gaia, but we could try to use the coordinates `ra` and `dec` to cross match.
We won't do that here, but it is common to have to do this sort of munging when dealing with interesting datasets.
For now we'll just remove those planets.

In [12]:
# we'll later join using loc_rowid
print(np.unique(nexsci["loc_rowid"]).size, len(nexsci))
# We don't expect the source IDs to be unique because there can be
# multiple planets per star
print(np.unique(nexsci["dr2_source_id"]).size, len(nexsci))
print(np.unique(nexsci["dr2_source_id"][nexsci["dr2_source_id"] != -1]).size, np.sum(nexsci["dr2_source_id"] != -1))

5044 5044
3609 5044
3608 4856


In [13]:
nexsci_coo = nexsci["loc_rowid","dr2_source_id","ra","dec"]

In [14]:
nexsci_coo.write("nexsci_coord.csv", overwrite=True)

Once uploaded, here is the query I did.

Note the table `gaiadr3.dr2_neighbourhood` which is the cross-match that Gaia did between DR2 and DR3. It is worth checking to see if this is a reasonable crossmatch.

Also note in some cases I did `LEFT JOIN`. This is because the number of sources in the NSS table is much less than the other tables, and I want to make sure all of them were kept.

```
SELECT
a.loc_rowid, a.dr2_source_id,
b.source_id, b.ra, b.dec, b.parallax, b.parallax_error, b.parallax_over_error,
b.pmra, b.pmra_error, b.pmdec, b.pmdec_error, b.ruwe, 
b.phot_g_mean_mag, b.phot_bp_mean_mag, b.phot_rp_mean_mag,
b.radial_velocity, b.radial_velocity_error,
b.rv_nb_transits, b.rv_expected_sig_to_noise, b.rv_renormalised_gof, b.rv_chisq_pvalue,
b.non_single_star, b.has_xp_continuous, b.has_rvs, b.has_epoch_photometry, b.has_epoch_rv,
b.teff_gspphot, b.logg_gspphot, b.mh_gspphot, b.distance_gspphot, b.ag_gspphot, b.ebpminrp_gspphot,
c.nss_solution_type, c.period, c.period_error, c.eccentricity, c.eccentricity_error, 
c.semi_amplitude_primary, c.semi_amplitude_primary_error, c.semi_amplitude_secondary, c.semi_amplitude_secondary_error,
c.mass_ratio, c.mass_ratio_error, c.goodness_of_fit, c.significance, c.g_luminosity_ratio,
d.teff_gspspec, d.logg_gspspec, d.mh_gspspec, d.alphafe_gspspec, d.flags_gspspec,
d.radius_flame, d.lum_flame, d.mass_flame, d.age_flame, d.flags_flame, d.evolstage_flame
FROM user_aji.nexsci as a
INNER JOIN
gaiadr3.dr2_neighbourhood as z ON a.dr2_source_id = z.dr2_source_id
INNER JOIN
gaiadr3.gaia_source_lite as b ON z.dr3_source_id = b.source_id
LEFT JOIN
gaiadr3.nss_two_body_orbit as c ON b.source_id = c.source_id
LEFT JOIN
gaiadr3.astrophysical_parameters as d ON b.source_id = d.source_id
```

In [15]:
gnexsci = Table.read("nexsci1-result.fits.gz")

In [16]:
len(gnexsci), len(nexsci_coo)

(5021, 5044)

We lost a few, it would be worth going back to see which ones and why.
But for now let's just see what we've got:

In [17]:
nexsciall = join(nexsci, gnexsci, "loc_rowid", "inner")

In [18]:
nexsciall

loc_rowid,pl_name,hostname,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_facility,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassprov,pl_dens,pl_denserr1,pl_denserr2,pl_denslim,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,ttv_flag,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_lum,st_lumerr1,st_lumerr2,st_lumlim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_age,st_ageerr1,st_ageerr2,st_agelim,st_dens,st_denserr1,st_denserr2,st_denslim,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,st_rotp,st_rotperr1,st_rotperr2,st_rotplim,st_radv,st_radverr1,st_radverr2,st_radvlim,ra_1,dec_1,sy_dist,sy_disterr1,sy_disterr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_jmag,sy_jmagerr1,sy_jmagerr2,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,dr2_source_id_1,dr2_source_id_2,source_id,ra_2,dec_2,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ruwe,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,radial_velocity,radial_velocity_error,rv_nb_transits,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,non_single_star,has_xp_continuous,has_rvs,has_epoch_photometry,has_epoch_rv,teff_gspphot,logg_gspphot,mh_gspphot,distance_gspphot,ag_gspphot,ebpminrp_gspphot,nss_solution_type,period,period_error,eccentricity,eccentricity_error,semi_amplitude_primary,semi_amplitude_primary_error,semi_amplitude_secondary,semi_amplitude_secondary_error,mass_ratio,mass_ratio_error,goodness_of_fit,significance,g_luminosity_ratio,teff_gspspec,logg_gspspec,mh_gspspec,alphafe_gspspec,flags_gspspec,radius_flame,lum_flame,mass_flame,age_flame,flags_flame,evolstage_flame
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,deg,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,km / s,km / s,,,,,,,,,,K,log(cm.s**-2),dex,pc,mag,mag,,d,d,,,km / s,km / s,km / s,km / s,,,,,,K,log(cm.s**-2),dex,dex,,solRad,solLum,solMass,Gyr,,
int64,str29,str27,str11,str12,str14,str28,int64,int64,int64,str29,int64,str178,str46,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,str16,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,int64,int64,str16,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,str6,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,int

Let's see all the ones which have non-single-star solutions in Gaia DR3!

In [19]:
nexscinss = nexsciall[nexsciall["non_single_star"]==1]

In [20]:
nexscinss

loc_rowid,pl_name,hostname,hd_name,hip_name,tic_id,gaia_id,sy_snum,sy_pnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_facility,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassprov,pl_dens,pl_denserr1,pl_denserr2,pl_denslim,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,ttv_flag,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_lum,st_lumerr1,st_lumerr2,st_lumlim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_age,st_ageerr1,st_ageerr2,st_agelim,st_dens,st_denserr1,st_denserr2,st_denslim,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,st_rotp,st_rotperr1,st_rotperr2,st_rotplim,st_radv,st_radverr1,st_radverr2,st_radvlim,ra_1,dec_1,sy_dist,sy_disterr1,sy_disterr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_jmag,sy_jmagerr1,sy_jmagerr2,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,dr2_source_id_1,dr2_source_id_2,source_id,ra_2,dec_2,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ruwe,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,radial_velocity,radial_velocity_error,rv_nb_transits,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,non_single_star,has_xp_continuous,has_rvs,has_epoch_photometry,has_epoch_rv,teff_gspphot,logg_gspphot,mh_gspphot,distance_gspphot,ag_gspphot,ebpminrp_gspphot,nss_solution_type,period,period_error,eccentricity,eccentricity_error,semi_amplitude_primary,semi_amplitude_primary_error,semi_amplitude_secondary,semi_amplitude_secondary_error,mass_ratio,mass_ratio_error,goodness_of_fit,significance,g_luminosity_ratio,teff_gspspec,logg_gspspec,mh_gspspec,alphafe_gspspec,flags_gspspec,radius_flame,lum_flame,mass_flame,age_flame,flags_flame,evolstage_flame
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,deg,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,km / s,km / s,,,,,,,,,,K,log(cm.s**-2),dex,pc,mag,mag,,d,d,,,km / s,km / s,km / s,km / s,,,,,,K,log(cm.s**-2),dex,dex,,solRad,solLum,solMass,Gyr,,
int64,str29,str27,str11,str12,str14,str28,int64,int64,int64,str29,int64,str178,str46,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,str16,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,int64,int64,str16,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,str6,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,float64,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,int

In [21]:
nexsciall.write("nexsci_gaia.csv", overwrite=True)

I don't know much about planets, but I hope this is a useful starting point!

# Try querying with astroquery

For a small list of source IDs, it should be possible to query the Gaia info from astroquery.
(It looks like the default limit with this method is 2000, but you can set it explicitly in your query.)

https://astroquery.readthedocs.io/en/latest/gaia/gaia.html

The good thing about this is that you don't have to make a Gaia account. The bad thing is that there's a smaller limit on what you can query, though anecdotally a few thousand objects is fine.

In [22]:
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"


In [23]:
# seems like vo table is a reasonable format to upload
nexsciall["loc_rowid","source_id"].write("nexsci_gaia_coord.vot", 
                                         overwrite=True, format="votable")

In [24]:
?Gaia.launch_job

In [25]:
# just querying ALL the columns
query ="""
SELECT g.*, nss.*
FROM gaiadr3.gaia_source as g
INNER JOIN tap_upload.nexsci as nexsci ON nexsci.source_id = g.source_id
INNER JOIN gaiadr3.nss_two_body_orbit as nss ON nss.source_id = g.source_id
"""
j = Gaia.launch_job(query=query,
                   upload_resource="nexsci_gaia_coord.vot",
                   upload_table_name="nexsci", verbose=True)
r = j.get_results()

Launched query: '
SELECT  TOP 2000 g.*, nss.*
FROM gaiadr3.gaia_source as g
INNER JOIN tap_upload.nexsci as nexsci ON nexsci.source_id = g.source_id
INNER JOIN gaiadr3.nss_two_body_orbit as nss ON nss.source_id = g.source_id
'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = multipart/form-data; boundary====1655180262517===
200 200
[('Date', 'Tue, 14 Jun 2022 04:17:43 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips PHP/5.4.16 mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Set-Cookie', 'JSESSIONID=8E4AB14EC0265894377A38121BE19726; Path=/tap-server; Secure; HttpOnly'), ('Content-Encoding', 'gzip'), ('Content-Disposition', 'attachment;filename="1655180263057O-result.vot"'), ('Transfer-Encoding', 'chunked'), ('Content-Type', 'application/x-votable

In [26]:
r

solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,solution_id_2,source_id_2,nss_solution_type,ra_2,ra_error_2,dec_2,dec_error_2,parallax_2,parallax_error_2,pmra_2,pmra_error_2,pmdec_2,pmdec_error_2,a_thiele_innes,a_thiele_innes_error,b_thiele_innes,b_thiele_innes_error,f_thiele_innes,f_thiele_innes_error,g_thiele_innes,g_thiele_innes_error,c_thiele_innes,c_thiele_innes_error,h_thiele_innes,h_thiele_innes_error,period,period_error,t_periastron,t_periastron_error,eccentricity,eccentricity_error,center_of_mass_velocity,center_of_mass_velocity_error,semi_amplitude_primary,semi_amplitude_primary_error,semi_amplitude_secondary,semi_amplitude_secondary_error,mass_ratio,mass_ratio_error,fill_factor_primary,fill_factor_primary_error,fill_factor_secondary,fill_factor_secondary_error,inclination,inclination_error,arg_periastron,arg_periastron_error,temperature_ratio,temperature_ratio_error,temperature_ratio_definition,astrometric_n_obs_al_2,astrometric_n_good_obs_al_2,rv_n_obs_primary,rv_n_good_obs_primary,rv_n_obs_secondary,rv_n_good_obs_secondary,phot_g_n_obs_2,phot_g_n_good_obs,bit_index,corr_vec,obj_func,goodness_of_fit,efficiency,significance,flags,conf_spectro_period,r_pole_sum,r_l1_point_sum,r_spher_sum,ecl_time_primary,ecl_time_secondary,ecl_dur_primary,ecl_dur_secondary,g_luminosity_ra